In [4]:
%run ../load_files.ipynb

In [9]:
%run ../data_dti_matrixes.ipynb

(140, 39)


In [10]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
01_SLD,0,0
02_MSG,0,0
03_JAC,0,0
04_NSV,0,0
05_EGR,0,0


In [11]:
y = y_values.memory

In [12]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.333, random_state=2018, stratify=y)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(93, 39)
(47, 39)
(93,)
(47,)


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', RandomForestClassifier())
])

In [15]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 500, 10)
MATRIXES = list(all_combinations(['DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX',]))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            
MAX_DEPTH = range(1, 100, 10)

param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__max_depth' : MAX_DEPTH,
}


In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [17]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=151, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__max_depth=31 
[CV] reduce_dim__n_components=151, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__max_depth=31 
[CV] reduce_dim__n_components=151, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__max_depth=31 
[CV] reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__max_depth=21 
[CV] reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__max_depth=21 
[CV] reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__max_depth=21 
[CV] reduce_dim__n_components=411, filter__columns=None, combining__columns=('DTI_RX',), classify__max_depth=1 
[CV] reduce_dim__n_components=411, filter__columns=None, combining__c

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.1s


[CV]  reduce_dim__n_components=221, filter__columns=None, combining__columns=('DTI_FA',), classify__max_depth=61, score=0.7419354838709677, total=   1.5s
[CV] reduce_dim__n_components=181, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__max_depth=1 
[CV]  reduce_dim__n_components=221, filter__columns=None, combining__columns=('DTI_FA',), classify__max_depth=61, score=0.7666666666666667, total=   1.6s
[CV] reduce_dim__n_components=181, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__max_depth=1 
[CV]  reduce_dim__n_components=271, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__max_depth=51, score=0.75, total=   1.8s
[CV]  reduce_dim__n_components=271, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__max_depth=51, score=0.7419354838709677, total=   1.5s
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'),

[CV] reduce_dim__n_components=451, filter__columns=None, combining__columns=('DTI_MD',), classify__max_depth=91 
[CV]  reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__max_depth=91, score=0.78125, total=   1.5s
[CV] reduce_dim__n_components=451, filter__columns=None, combining__columns=('DTI_MD',), classify__max_depth=91 
[CV]  reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__max_depth=91, score=0.7419354838709677, total=   1.3s
[CV] reduce_dim__n_components=11, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__max_depth=31 
[CV]  reduce_dim__n_components=331, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__max_depth=61, score=0.75, total=   1.7s
[CV] reduce_dim__n_components=11, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__max_depth=31 
[CV]  reduce_dim__n_components=421, filter__columns=None, combining_

[CV]  reduce_dim__n_components=61, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__max_depth=91, score=0.7666666666666667, total=   1.8s
[CV]  reduce_dim__n_components=111, filter__columns=None, combining__columns=('DTI_MD',), classify__max_depth=21, score=0.7419354838709677, total=   1.5s
[CV]  reduce_dim__n_components=111, filter__columns=None, combining__columns=('DTI_MD',), classify__max_depth=21, score=0.75, total=   1.7s
[CV] reduce_dim__n_components=121, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__max_depth=61 
[CV]  reduce_dim__n_components=111, filter__columns=None, combining__columns=('DTI_MD',), classify__max_depth=21, score=0.7666666666666667, total=   1.4s
[CV] reduce_dim__n_components=121, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__max_depth=61 
[CV] reduce_dim__n_components=371, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__max_depth=11 
[CV] reduce_di

...

[CV] reduce_dim__n_components=211, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__max_depth=91 
[CV]  reduce_dim__n_components=311, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__max_depth=31, score=0.7419354838709677, total=   1.6s
[CV] reduce_dim__n_components=211, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__max_depth=91 
[CV]  reduce_dim__n_components=441, filter__columns=None, combining__columns=('DTI_FA',), classify__max_depth=21, score=0.75, total=   1.7s
[CV] reduce_dim__n_components=211, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__max_depth=91 
[CV]  reduce_dim__n_components=441, filter__columns=None, combining__columns=('DTI_FA',), classify__max_depth=21, score=0.7419354838709677, total=   1.5s
[CV] reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__max_depth=51 
[CV]  reduce_dim__n_components=441, filter__columns=Non

[CV] reduce_dim__n_components=101, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), classify__max_depth=81 
[CV]  reduce_dim__n_components=71, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__max_depth=21, score=0.7666666666666667, total=   1.2s
[CV]  reduce_dim__n_components=211, filter__columns=None, combining__columns=('DTI_RX',), classify__max_depth=1, score=0.7666666666666667, total=   2.0s
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_RX',), classify__max_depth=71 
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_RX',), classify__max_depth=71 
[CV]  reduce_dim__n_components=71, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__max_depth=21, score=0.7419354838709677, total=   1.6s
[CV]  reduce_dim__n_components=71, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__max_depth=21, score=0.75, t

[CV]  reduce_dim__n_components=151, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__max_depth=51, score=0.7666666666666667, total=   1.2s


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  2.6min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7f18c019f510>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=None, matrix_columns=['combined'])), ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_MD', 'DTI_RX'), ('DTI_FA', 'DTI_L1', 'DTI_MD'), ('DTI_FA', 'DTI_L1', 'DTI_RX'), ('DTI_FA...s': [None], 'reduce_dim__n_components': range(1, 500, 10), 

In [18]:
%run ../grid_results.ipynb

reduce_dim__n_components = 121
filter__columns = None
combining__columns = ('DTI_MD', 'DTI_RX')
classify__max_depth = 81
Test
Accuracy in test: 74.47%
             precision    recall  f1-score   support

          0       0.74      1.00      0.85        35
          1       0.00      0.00      0.00        12

avg / total       0.55      0.74      0.64        47

Train


/home/efrain/.virtualenvs/neudataload/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy in train: 93.55%
             precision    recall  f1-score   support

          0       0.92      1.00      0.96        70
          1       1.00      0.74      0.85        23

avg / total       0.94      0.94      0.93        93

